# Building the game with proper algo implementations

In [1]:
!pip install easyAI

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.5 MB/s eta 0:00:00


## Building Unbeatable AI with alpha beta pruning on minmax for tictactoe

In [9]:
from easyAI import TwoPlayerGame
from easyAI.Player import Human_Player, AI_Player
import math

class TicTacToe(TwoPlayerGame):
    """The board positions are numbered as follows:
    1 2 3
    4 5 6
    7 8 9
    """

    def __init__(self, players=None):
        self.players = players
        self.board = [0 for _ in range(9)]
        self.current_player = 1  # player 1 starts.

    def possible_moves(self):
        return [i + 1 for i, e in enumerate(self.board) if e == 0]

    def make_move(self, move):
        self.board[int(move) - 1] = self.current_player

    def unmake_move(self, move):  # optional method (speeds up the AI)
        self.board[int(move) - 1] = 0

    def lose(self):#will tell if the opponent has won considering the current index
        """Has the opponent "three in line"?"""
        return any(
            all([(self.board[c - 1] == self.opponent_index) for c in line])
            for line in [
                [1, 2, 3], [4, 5, 6], [7, 8, 9],  # horizontal
                [1, 4, 7], [2, 5, 8], [3, 6, 9],  # vertical
                [1, 5, 9], [3, 5, 7]  # diagonal
            ]
        )

    def is_over(self):
        return (self.possible_moves() == []) or self.lose()

    def show(self):
        print(
            "\n"
            + "\n".join(
                [
                    " ".join([[".", "O", "X"][self.board[3 * j + i]] for i in range(3)])
                    for j in range(3)
                ]
            )
        )


    def minmax(self, alpha, beta, maximizing):
      if self.lose():
        return -1 if maximizing else 1 #if maximising(AI turn) and opponent wins return -1 as bad for ai else(then human turn) and opponent(ai wins) then return 1 as good for ai
      elif self.is_over():
          return 0 #if draw then return 0

      if maximizing:
          best_score = -math.inf #since maximising so init with -inf
          for move in self.possible_moves():
              self.make_move(move)
              self.current_player = 3 - self.current_player  # Switch turn(since we have human with value 1 and AI with value 2 to switch we can do this directly)
              score = self.minmax(alpha, beta, False)
              self.unmake_move(move)
              self.current_player = 3 - self.current_player  # Restore turn
              best_score = max(best_score, score)
              alpha = max(alpha, best_score)
              if beta <= alpha:
                  break  # Prune
          return best_score
      else:
          best_score = math.inf # since minimising so init with inf
          for move in self.possible_moves():
              self.make_move(move)
              self.current_player = 3 - self.current_player  # Switch turn
              score = self.minmax(alpha, beta, True)
              self.unmake_move(move)
              self.current_player = 3 - self.current_player  # Restore turn
              best_score = min(best_score, score)
              beta = min(beta, best_score)
              if beta <= alpha:
                  break  # Prune
          return best_score
    '''
    so here also we get it after root node and we make move for the node and get score with all other nodes with switch of player and also with switch  of max or min and based
    node we are in like if max node then take max of score with current alpha and prune is done if beta <= alpha(both cases)
    '''

    def ai_move(self):
      best_score = -math.inf
      best_move = None
      for move in self.possible_moves():
          self.make_move(move)
          self.current_player = 3 - self.current_player  # Switch turn
          score = self.minmax(-math.inf, math.inf, False)
          self.unmake_move(move)
          self.current_player = 3 - self.current_player  # Restore turn

          if score > best_score:
              best_score = score
              best_move = move  # Fix: Only update best_move if score is higher
      return best_move
      '''
      key idea is that the ai_move is the main root node which is maximum so for all moves as root node it calls the min max and among all takes the best score and gives
      and since here move is made we send as minimising that is it is the opponent turn so we switch the player and we return the best move
      and always alpha is -inf as max we take and beta is +inf as we take min
      '''

    def play(self):
        print("Welcome to Tic-Tac-Toe!")
        print("Player 1 (Human) is 'X' and Player 2 (AI) is 'O'.")
        self.show()

        while not self.is_over():
            if self.current_player == 1:
                move = self.players[0].ask_move(self)
            else:
                move = self.ai_move()
                print(f"AI chooses: {move}")

            self.make_move(move)
            self.show()
            self.current_player = 3 - self.current_player  # Switch player

        if self.lose():
            print(f"Player {3 - self.current_player} wins!")#since at end we switch players and if game is over then the switched player is loser and switch player again to get winner
        else:
            print("It's a draw!")

# Start the game
game = TicTacToe([Human_Player(), AI_Player(None)])  # Human plays, AI uses Minimax
game.play()


Welcome to Tic-Tac-Toe!
Player 1 (Human) is 'X' and Player 2 (AI) is 'O'.

. . .
. . .
. . .

Player 1 what do you play ? 1

O . .
. . .
. . .
AI chooses: 5

O . .
. X .
. . .

Player 1 what do you play ? 2

O O .
. X .
. . .
AI chooses: 3

O O X
. X .
. . .

Player 1 what do you play ? 4

O O X
O X .
. . .
AI chooses: 7

O O X
O X .
X . .
Player 2 wins!


## Making the tictactoe with iterative deepening with minmax(not always optimal guaranteed)

In [14]:
import time

class TicTacToe(TwoPlayerGame):
    def __init__(self, players=None):
        self.players = players
        self.board = [0 for _ in range(9)]
        self.current_player = 1

    def possible_moves(self):
        return [i + 1 for i, e in enumerate(self.board) if e == 0]

    def make_move(self, move):
        self.board[int(move) - 1] = self.current_player

    def unmake_move(self, move):
        self.board[int(move) - 1] = 0

    def lose(self):
        return any(
            all([(self.board[c - 1] == self.opponent_index) for c in line])
            for line in [[1, 2, 3], [4, 5, 6], [7, 8, 9], [1, 4, 7], [2, 5, 8], [3, 6, 9], [1, 5, 9], [3, 5, 7]]
        )

    def is_over(self):
        return self.possible_moves() == [] or self.lose()

    def show(self):
        print(
            "\n"
            + "\n".join(
                [
                    " ".join([[".", "O", "X"][self.board[3 * j + i]] for i in range(3)])
                    for j in range(3)
                ]
            )
        )
    def minmax(self, alpha, beta, maximizing, depth):
        if self.lose():
            return -1 if maximizing else 1
        elif self.is_over():
            return 0
        elif depth == 0:  # Stop at max depth
            return 0  # Can be replaced with a heuristic function

        if maximizing:
            best_score = -math.inf
            for move in self.possible_moves():
                self.make_move(move)
                self.current_player = 3 - self.current_player
                score = self.minmax(alpha, beta, False, depth - 1)
                self.unmake_move(move)
                self.current_player = 3 - self.current_player
                best_score = max(best_score, score)
                alpha = max(alpha, best_score)
                if beta <= alpha:
                    break
            return best_score
        else:
            best_score = math.inf
            for move in self.possible_moves():
                self.make_move(move)
                self.current_player = 3 - self.current_player
                score = self.minmax(alpha, beta, True, depth - 1)
                self.unmake_move(move)
                self.current_player = 3 - self.current_player
                best_score = min(best_score, score)
                beta = min(beta, best_score)
                if beta <= alpha:
                    break
            return best_score

    def ai_move(self):
        start_time = time.time()
        max_depth = 1
        best_move = None

        while time.time() - start_time < 2:  # Set time limit (e.g., 2 seconds)
            best_score = -math.inf
            for move in self.possible_moves():
                self.make_move(move)
                self.current_player = 3 - self.current_player
                score = self.minmax(-math.inf, math.inf, False, max_depth)
                self.unmake_move(move)
                self.current_player = 3 - self.current_player

                if score > best_score:
                    best_score = score
                    best_move = move  # Store the best move found at this depth

            max_depth += 1  # Increase search depth
            if best_score == 1:  # If AI finds a winning move, stop searching
                break

        return best_move
      #basically just depth move on increase until best move got mainly only certain depth we create and move(here time taken)

    def play(self):
      print("Welcome to Tic-Tac-Toe!")
      print("Player 1 (Human) is 'X' and Player 2 (AI) is 'O'.")
      self.show()

      while not self.is_over():
          if self.current_player == 1:
              move = self.players[0].ask_move(self)
          else:
              move = self.ai_move()
              print(f"AI chooses: {move}")

          self.make_move(move)
          self.show()
          self.current_player = 3 - self.current_player  # Switch player

      if self.lose():
          print(f"Player {3 - self.current_player} wins!")#since at end we switch players and if game is over then the switched player is loser and switch player again to get winner
      else:
          print("It's a draw!")

game = TicTacToe([Human_Player(), AI_Player(None)])  # Human plays, AI uses Minimax
game.play()

'''
Iterative Deepening does not guarantee the absolute best move unless it reaches the full game depth.

If the winning move is found early, it works great. But if the best move requires deeper exploration, it might miss it if stopped too soon.

So, it’s a trade-off:
✅ Good if stopped at the right time
❌ Not guaranteed best if depth limit is too low
'''

#since game is tictactoe and doesnt have too much depth we can expect this also to be unbeatable but for games like chess with very large depth on restrictions we play with optimality then

Welcome to Tic-Tac-Toe!
Player 1 (Human) is 'X' and Player 2 (AI) is 'O'.

. . .
. . .
. . .

Player 1 what do you play ? 1

O . .
. . .
. . .
AI chooses: 5

O . .
. X .
. . .

Player 1 what do you play ? 3

O . O
. X .
. . .
AI chooses: 2

O X O
. X .
. . .

Player 1 what do you play ? 8

O X O
. X .
. O .
AI chooses: 4

O X O
X X .
. O .

Player 1 what do you play ? 6

O X O
X X O
. O .
AI chooses: 9

O X O
X X O
. O X

Player 1 what do you play ? 7

O X O
X X O
O O X
It's a draw!
